In [1]:
from obspy import read
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.stats import mode, skew, kurtosis, zscore

# Function to sanitize file names
def sanitize_filename(name):
    return name.replace(".", "").replace(":", "").replace("-", "").replace("_", "")

# Function to convert ADX355 counts to acceleration
def convert_adx355_counts_to_acceleration(counts, scale_factor=256000):
    acceleration_g = counts / scale_factor  # Convert counts to g
    acceleration_m_s2 = acceleration_g * 9.81  # Convert g to m/s²
    return acceleration_g, acceleration_m_s2

def compute_mer(signal, window_size=50):
    energy = np.convolve(signal**2, np.ones(window_size), mode='valid')
    mer = np.zeros(len(signal))
    mer[window_size - 1: window_size - 1 + len(energy)] = energy
    return mer

# Function to compute STA/LTA ratio
def compute_sta_lta(signal, sta_window=50, lta_window=200):
    sta = np.convolve(signal, np.ones(sta_window) / sta_window, mode="valid")
    lta = np.convolve(signal, np.ones(lta_window) / lta_window, mode="valid")

    sta_lta_ratio = np.zeros(len(signal))
    valid_length = min(len(sta), len(lta))
    sta_lta_ratio[lta_window - 1: lta_window - 1 + valid_length] = sta[:valid_length] / (lta[:valid_length] + 1e-10)

    return sta_lta_ratio

# Directories
mseed_dir = "miniSEED_files"
output_csv = "acceleration_stats_1.5sec_windows_separated_28columns.csv"

# Number of samples per window
sampling_rate = 250  # 250 Hz means 250 samples per second
num_samples_before = sampling_rate  # 1 second window (Before)
num_samples_after = 125  # 0.5 second window (After)

# STA/LTA window lengths
sta_window_before = 50  # Short-term average window for before
lta_window_before = 200  # Long-term average window for before

sta_window_after = 25  # Short-term average window for after
lta_window_after = 100  # Long-term average window for after

# Store results
results = []

# Stats headers
stats_headers = ["Mean", "Median", "Mode", "Std Dev", "Skewness", "Kurtosis", "Variance", "Max", "Min", "Z-Score", 
                 "Mean STA/LTA", "Max STA/LTA", "Mean MER", "Max MER"]

# Create column headers
columns = ["File", "Trace ID", "Window Start (sec)"]

# Add columns for each of the stats for "Before" and "After" windows
for sample in ["Before", "After"]:
    columns.extend([f"{sample} {stat}" for stat in stats_headers])

# Loop through miniSEED files
for file_path in Path(mseed_dir).glob("*.MSEED"):
    mseed_name = sanitize_filename(file_path.name.strip())

    print(f"Processing: {file_path.name}")

    # Read miniSEED file
    stream = read(file_path)

    for trace in stream:
        print(f"Processing trace: {trace.id}")

        total_samples = len(trace.data)

        try:
            # Convert counts to acceleration
            _, accel_m_s2 = convert_adx355_counts_to_acceleration(trace.data)
        except Exception as e:
            print(f"Error converting counts for {trace.id}: {e}")
            continue

        # Get time array
        times = np.linspace(trace.stats.starttime.timestamp, trace.stats.endtime.timestamp, total_samples)

        # Compute STA/LTA ratio for both before and after
        sta_lta_ratio_before = compute_sta_lta(accel_m_s2, sta_window=sta_window_before, lta_window=lta_window_before)
        sta_lta_ratio_after = compute_sta_lta(accel_m_s2, sta_window=sta_window_after, lta_window=lta_window_after)

        # Loop for each 1.5 second window (1 second before, 0.5 second after) with 1-second overlap
        for window_start in range(0, total_samples - num_samples_before - num_samples_after, num_samples_before):  # Shift by 1 sec for overlap
            # Define the indices for the current window
            start_idx_before = window_start
            end_idx_before = start_idx_before + num_samples_before

            start_idx_after = window_start + num_samples_before
            end_idx_after = start_idx_after + num_samples_after

            # Extract segments for before and after windows
            segment_before = accel_m_s2[start_idx_before:end_idx_before]
            segment_after = accel_m_s2[start_idx_after:end_idx_after]
            sta_lta_segment_before = sta_lta_ratio_before[start_idx_before:end_idx_before]
            sta_lta_segment_after = sta_lta_ratio_after[start_idx_after:end_idx_after]
            mer_segment_before = compute_mer(segment_before)  # Compute MER for the before segment
            mer_segment_after = compute_mer(segment_after)  # Compute MER for the after segment

            # Compute statistics for before window
            mean_before = np.mean(segment_before)
            median_before = np.median(segment_before)
            mode_before = mode(segment_before, keepdims=True).mode[0]
            std_before = np.std(segment_before)
            skewness_before = skew(segment_before)
            kurtosis_before = kurtosis(segment_before)
            variance_before = np.var(segment_before)
            max_before = np.max(segment_before)
            min_before = np.min(segment_before)
            z_scores_before = np.mean(zscore(segment_before))
            mean_sta_lta_before = np.mean(sta_lta_segment_before)
            max_sta_lta_before = np.max(sta_lta_segment_before)
            mean_mer_before = np.mean(mer_segment_before)
            max_mer_before = np.max(mer_segment_before)

            # Compute statistics for after window
            mean_after = np.mean(segment_after)
            median_after = np.median(segment_after)
            mode_after = mode(segment_after, keepdims=True).mode[0]
            std_after = np.std(segment_after)
            skewness_after = skew(segment_after)
            kurtosis_after = kurtosis(segment_after)
            variance_after = np.var(segment_after)
            max_after = np.max(segment_after)
            min_after = np.min(segment_after)
            z_scores_after = np.mean(zscore(segment_after))
            mean_sta_lta_after = np.mean(sta_lta_segment_after)
            max_sta_lta_after = np.max(sta_lta_segment_after)
            mean_mer_after = np.mean(mer_segment_after)
            max_mer_after = np.max(mer_segment_after)

            # Append stats for both before and after windows
            results.append([file_path.name, trace.id, window_start, 
                            mean_before, median_before, mode_before, std_before, 
                            skewness_before, kurtosis_before, variance_before, 
                            max_before, min_before, z_scores_before, mean_sta_lta_before, 
                            max_sta_lta_before, mean_mer_before, max_mer_before, 
                            mean_after, median_after, mode_after, std_after, 
                            skewness_after, kurtosis_after, variance_after, 
                            max_after, min_after, z_scores_after, mean_sta_lta_after, 
                            max_sta_lta_after, mean_mer_after, max_mer_after])

# Save results to Excel
df_results = pd.DataFrame(results, columns=columns)
df_results.to_csv(output_csv, index=False)
print(f"Acceleration statistics with STA/LTA and MER saved to {output_csv}")


Processing: 34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DN1.MSEED
Processing trace: WS.POZA.S5.DN1
Processing: 34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DN2.MSEED
Processing trace: WS.POZA.S5.DN2
Processing: 34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DNZ.MSEED
Processing trace: WS.POZA.S5.DNZ
Processing: 34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DN1.MSEED
Processing trace: WS.POZA.S3.DN1
Processing: 34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DN2.MSEED
Processing trace: WS.POZA.S3.DN2
Processing: 34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DNZ.MSEED
Processing trace: WS.POZA.S3.DNZ
Processing: 34161341_2023-02-21T00.07.00.490032Z_WS.POZA.S2.DN1.MSEED
Processing trace: WS.POZA.S2.DN1
Processing: 34161341_2023-02-21T00.07.00.490032Z_WS.POZA.S2.DN2.MSEED
Processing trace: WS.POZA.S2.DN2
Processing: 34161341_2023-02-21T00.07.00.490032Z_WS.POZA.S2.DNZ.MSEED
Processing trace: WS.POZA.S2.DNZ
Processing: 34161341_2023-02-21T00.07.00.490708Z_WS.POZA.S4.DN1.MSEED
Pro